In [10]:
#Import all necessary packages
import numpy as np
import pandas as pd
import os, shutil,cv2,imutils
import matplotlib.image as mpimg
# Use "conda install -c conda-forge scikit-image" incase skimage install throws error
from skimage.filters import threshold_local
import requests, re
# If you are using a Jupyter notebook, uncomment the following line.
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from io import BytesIO

In [11]:
# Class for the different shapes and pattern match

class ShapeDetector:
    def __init__(self):
        pass

    def detect(self, c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)

        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        if len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)

            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"

        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"

        # return the name of the shape
        return shape
        
# Class for matching different date format
class DateMatch:
    
    def __init__(self):
        pass
    
    def dateandtime_match(self,text):
        
        match = re.search(r'(\d+/\d+/\d+)',text)
        return None if match is None else match is True
        match = re.search(r'(^[0-3]?[0-9]/[0-3]?[0-9]/(?:[0-9]{2})?[0-9]{2}$)',text)
        return None if match is None else match is True
        match = re.search(r'(^[0-3][0-9]/[0-3][0-9]/(?:[0-9][0-9])?[0-9][0-9]$)',text)
        return None if match is None else match is True
        match = re.search(r'(^(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])/(?:[0-9]{2})?[0-9]{2}$)',text)
        return None if match is None else match is True
        match = re.search(r'(^(1[0-2]|0[1-9])/(3[01]|[12][0-9]|0[1-9])/[0-9]{4}$)',text)
        return None if match is None else match is True
        match = re.search(r'(^(3[01]|[12][0-9]|0?[1-9])/(1[0-2]|0?[1-9])/(?:[0-9]{2})?[0-9]{2}$)',text)
        return None if match is None else match is True
        match = re.search(r'(^(3[01]|[12][0-9]|0[1-9])/(1[0-2]|0[1-9])/[0-9]{4}$)',text)
        return None if match is None else match is True
        match = re.search(r'(^(?:(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])|(3[01]|[12][0-9]|0?[1-9])/(1[0-2]|0?[1-9]))/(?:[0-9]{2})?[0-9]{2}$)',text)
        return None if match is None else match is True
        match = re.search(r'(^(?:(1[0-2]|0?[1-9])/(3[01]|[12][0-9]|0?[1-9])|(3[01]|[12][0-9]|0?[1-9])/(1[0-2]|0?[1-9]))/(?:[0-9]{2})?[0-9]{2}$)',text)
        return None if match is None else match is True
        match = re.search(r'(^(?:(1[0-2]|0[1-9])/(3[01]|[12][0-9]|0[1-9])|(3[01]|[12][0-9]|0[1-9])/(1[0-2]|0[1-9]))/[0-9]{4}$)',text)
        return None if match is None else match is True
        match = re.search(r'(^(?:(1[0-2]|0[1-9])/(3[01]|[12][0-9]|0[1-9])|(3[01]|[12][0-9]|0[1-9])/(1[0-2]|0[1-9]))/[0-9]{4}$)',text)
        return None if match is None else match is True
        
        return match


# Class for matching common words in receipts
class CommonWords:
    
    def __init__(self):
        pass
    
    def word_match(self,text):
        
        patterns = ['Total','TOTAL','total','Amount','amount','AMOUNT','Cash','cash','CASH','Thank You','thank you','Thank you',
                    'THANK YOU','Thank You!','thank you!','Thank you!','THANK YOU!','card','Card','CARD','Debit','debit','DEBIT',
                    'Credit','credit','CREDIT','Item','item','ITEM','QTY','qty','QUANTITY','quantity','Quantity','TAX','Tax','tax',
                    'Receipt','receipt','RECEIPT','Balance','balance','BALANCE','Price','price','PRICE','Customer','customer','CUSTOMER',
                    'Merchant','merchant','MERCHANT','Items','items','ITEMS','@','.com','.COM','email','EMAIL',':','Please','please',
                    'PLEASE','Welcome','welcome','WELCOME','thanks','Thanks','THANKS','MEAL','meal','Meal','chng','Chng','CHNG'
                    '#','-','Call','call','CALL','again','Again','AGAIN','&','Transaction','transaction','TRANSACTION','$',
                    'subtotal','Subtotal','SUBTOTAL']
        
        wordcount = 0
        # Match for the words
        for pattern in patterns:
            if re.search(pattern,text):
                wordcount +=1
                       
        return wordcount       

In [12]:
# Functions for fourier transform

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4,2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped

In [13]:
def shapeDetect(image):

    # load the image and compute the ratio of the old height
    # to the new height, clone it, and resize it
    image = mpimg.imread(image)
    ratio = image.shape[0] / 500.0
    orig = image.copy()
    fortext = "Original.png"
    cv2.imwrite(fortext,image)
    image = imutils.resize(image, height = 500)

    # convert the image to grayscale, blur it, and find edges in the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 75, 200)

    # show the original image and the edge detected image 
    # find the contours in the edged image, keeping only the
    # largest ones, and initialize the screen contour
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # if our approximated contour has four points, then we can assume that we have found our screen
        if len(approx) == 4:
            screenCnt = approx
            # show the contour (outline) of the piece of paper
            cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
            #print("Yes!, it has four edges")
            break
            
    #Save the image to project directory
    edged = "{}.png".format(os.getpid())
    cv2.imwrite(edged,image)
    
    """
    #Read the edged Image to detect the shape
    image = cv2.imread(edged)
    #orig = image.copy()
    resized = imutils.resize(image, width=300)
    ratio_ = image.shape[0] / float(resized.shape[0])
    #os.remove(edged)
    
    # convert the resized image to grayscale, blur it slightly,
    # and threshold it
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)[1]
    
    # find contours in the thresholded image and initialize the
    # shape detector
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    sd = ShapeDetector()

   
    # loop over the contours
    for c in cnts:
        # compute the center of the contour, then detect the name of the
        # shape using only the contour
        M = cv2.moments(c)
        if M["m00"] != 0:
            cX = int((M["m10"] / M["m00"]) * ratio_)
            cY = int((M["m01"] / M["m00"]) * ratio_)
            shape = sd.detect(c)
            if shape == "rectangle":
                receipt = True
                break
            else:
                receipt = False
               
        else:
        
            font = cv2.FONT_HERSHEY_COMPLEX
            approx = cv2.approxPolyDP(c, 0.01*cv2.arcLength(c, True), True)
            x = approx.ravel()[0]
            y = approx.ravel()[1]
            if len(approx) == 4:
                receipt = True
                break
            else:
                receipt = False
            
    if receipt == True:
        # Add part here to scan the image and apply OCR on it instead of print statement
        print("Yes!, it has four edge")
    else:
        print("No! not with four edge")
    
    """
    os.remove(edged)
    # apply the four point transform to obtain a top-down
    # view of the original image
    warped = four_point_transform(orig, screenCnt.reshape(4,2) * ratio)
 
    # convert the warped image to grayscale, then threshold it
    # to give it that 'black and white' paper effect
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    T = threshold_local(warped,11, offset = 10, method = "mean")
    warped = (warped > T).astype("uint8") * 255
 
    # show the original and scanned images
    output = "Output.png"
    cv2.imwrite(output, imutils.resize(warped, height = 650))   
    
    # Text detection
    # Replace <Subscription Key> with your valid subscription key.
    subscription_key = "3c1471dee9084c4587fc64cbb0d225dd"
    assert subscription_key

    # You must use the same region in your REST call as you used to get your subscription keys. For example, if you got your 
    #subscription keys from westus, replace "westcentralus" in the URI below with "westus".Free trial subscription keys are generated in the "westus" region.
    # If you use a free trial subscription key, you shouldn't need to change this region.

    vision_base_url = "https://westus.api.cognitive.microsoft.com/vision/v2.0/"
    ocr_url = vision_base_url + "ocr"

    # Set image_url to the URL of an image that you want to analyze.

    #file_name = orig.copy()
    with open(output, 'rb') as f:
        img_data = f.read()

    headers = {'Content-Type': 'application/octet-stream','Ocp-Apim-Subscription-Key': subscription_key}
    params = {'language': 'unk', 'detectOrientation': 'true'}
    response = requests.post(ocr_url,headers=headers,params=params,data=img_data)
    response.raise_for_status()

    response
    analysis = response.json()

    # Extract the word bounding boxes and text.
    line_infos = [region["lines"] for region in analysis["regions"]]
    word_infos = []
    for line in line_infos:
        for word_metadata in line:
            for word_info in word_metadata["words"]:
                word_infos.append(word_info)

    for word in word_infos:
        bbox = [int(num) for num in word["boundingBox"].split(",")]
        text = word["text"]
        origin = (bbox[0], bbox[1])
        patch = Rectangle(origin, bbox[2], bbox[3],fill=False, linewidth=2, color='y')

    text = ''
    for item in response.json()['regions']:
        for line in item['lines']:
            for word in line['words']:
                text +=' ' + word['text']
            text += '\n'
    data = open("text.txt", 'w+')
    data.write(text)
    data.close()
    
    # Open the text file and read the content for the patterns matching
    data1 = open("text.txt", 'r')
    text1 = data1.read()
    dm = DateMatch()
    pattern1 = dm.dateandtime_match(text1)
    cw = CommonWords()
    pattern2 = cw.word_match(text1)
    data1.close()
    os.remove("text.txt")
    os.remove(output)
    os.remove(fortext)
    
    # Moment of the truth
    if pattern1 == True or pattern2 >= 2:
        #print("Yes! It is a receipt")
        isreceipt = True
    else:
        #print("Sorry! Please upload a receipt")
        isreceipt = False

    return isreceipt

#shapeDetect("b.jpeg")


True